# Data Science PI7, Part 1

### 1. SQL Database Analysis

To assess the available values in our database ("cycling_big.db"), we will have to explore our database.
We will import the necessary modules and then make a query to our database by calling //the names of the tables and their column values//

Note that these operations are better handled by data handling and manipulation modules, like Pandas, which will be used later on.

In [1]:
import sqlite3 as lite
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import numpy as np

In [ ]:
conn = lite.connect('cycling_big.db')
cur = conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

print("The following tables are present in our database")
for row in tables:
    print(row)

# We will be closing the database connection at the end of each code cell.
# By doing this we can make sure that queries dont interact with unwanted code cells.
conn.close()

Note that the tables 'riders' and 'race_results' are present in the database. 
With this information, a query can be made to print out the first ten rows from our tables.

In [ ]:
conn = lite.connect('cycling_big.db')
cur = conn.cursor()

conn = lite.connect('cycling_big.db')

# Execute a query to select all entries and put the labels in an arrary of description values

cur.execute('SELECT * FROM race_results')

columns_race = [description[0] for description in cur.description]

print("Columns for race_results:", columns_race)

cur.execute('SELECT * FROM riders')

columns_rider = [description[0] for description in cur.description]

print("Columns for riders:", columns_rider)

conn.close()

#### Evaluating empty values quickly

Now let's quickly identify the amount of empty cells for race results. 
This will not give a full scope of missing or incorrect data, but this will offer us a quick overview of the columns that are missing entries.
Ofcourse, entry values like 'Date' can be expected to be unstandardized, or a value can be entered as 'null', but will still count as a value, so using SQL queries will not be the most accurate to determine missing or unstandardized data.

In [ ]:
conn = lite.connect('cycling_big.db')
cur = conn.cursor()

cur.execute("PRAGMA table_info(race_results);") # Quick and dirty PRAGMA statement to return info on a column
columns = cur.fetchall()

# Building an SQL query to loop through all the columns to count nulls
null_count = "SELECT "

# Add each column's NULL count
null_count += ", ".join([f"SUM(CASE WHEN {column[1]} IS NULL THEN 1 ELSE 0 END) AS {column[1]}_null" for column in columns])

null_count += " FROM race_results;"

# Execute the query
cur.execute(null_count)

# Fetch the result
null_counts = cur.fetchone()

print("\nMissing Values Count for Each Column:")
print("-" * 50)
print(f"{'Column Name':<30} {'Missing Values'}")
print("-" * 50)

# Looping through the columns and applying formatting
for i, column in enumerate(columns):
    column_name = column[1]  # Column name is the second element in the tuple
    missing_values = null_counts[i]
    print(f"{column_name:<30} {missing_values:>15}")

print("-" * 50)

conn.close()

### 2. Database Extraction and Deserialization

Below is a code sample for extracting data from a SQLite database. Using the built in 'sqlite' module for python, we can open a connection by calling the 'connect()' function that points to a "*.db" file. The returned connection object, named 'conn', represents the connection to the on-disk database. This essentially creates an interface for python to interact with the SQLite database. Using the 'Pandas' module, we can put this deserialized data from the database into a dataframe. A dataframe preserves the rows and columns from the relevant database. By using this data frame, the data from the database can be used for data handling, manipulation and further analysis.

In [ ]:
# As shown in the above cells, a connection to the ".db" file can be established by 
conn = lite.connect("cycling_big.db")

table_names = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn) # Sqlite master is a system table that holds information like table names 
df = pd.read_sql(f"SELECT * FROM {table_names['name'][0]}", conn) # Access the first tables name from the table_names dataframe

conn.close()

df.head()

### 3. Exploratory Data Analysis

Exploratory analysis can be used to identify extreme cases and determine wether they should be eliminated.

- Boxplots will be made to identify outliers in the following features: ///. This is also useful for unsupervised learning, since a boxplot can display relations between a numerical and a categorical value.
- Scatterplots will be made to identify relations between the following continuous variables: age vs ranking and weight vs race position.
- A histogram will be made to analyze the distribution of numeric data for the distribution of points among cyclists.
- Heatmap to analyse correlation between variables

In [ ]:
conn = lite.connect('cycling_big.db')

riders_df = pd.read_sql_query('SELECT * FROM riders;', conn)
races_df = pd.read_sql_query('SELECT * FROM race_results', conn)

# Combine the tables so we can use variables from the two tables for analysis
# We attach rider information to races table, so that every entry in the races will have additional information on the rider
# rider_id is the key connecting the tables
df = races_df.set_index('rider_id').join(riders_df.set_index('rider_id'))

conn.close()
df.head(3)

In [ ]:
df.info()


In [ ]:
sns.boxplot(data=df['Age'])
plt.xlabel("")
plt.ylabel("Age")

In [ ]:
relevant_columns = ['Age', 'weight', 'height', 'Pnt', 'UCI', 'Stage_Type']
df_clean = df[relevant_columns].dropna()
# Create pairplot to visualize relationships between numeric columns
sns.pairplot(df_clean[relevant_columns], kind='scatter', plot_kws={'alpha': 0.6}, hue='Stage_Type')
plt.suptitle("Pairplot of Cyclists' Attributes", y=1.02)
plt.show()

In [ ]:
# Unless data gets cleaned/formatted (which we will be doing later), only numerical data can be used

relevant_columns = ['Age', 'weight', 'height', 'Pnt', 'UCI']
df_clean = df[relevant_columns].dropna()

corr = df_clean.corr()
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, vmin=-1, vmax=1, cmap="RdBu")

### 4. Quality of the Data

The quality of the data will be identified by the following characteristics:
- Completeness: this will indicated if the database is missing any entries. This can be achieved by finding null values. Missing data may also be annotated in a different manner; for example with '-' or '0'.
- Consistency: this will indicate if the formatting of the data is uniform or not. Think of time/day formats, which can variate among eachother. Identifying consistency in data will result in more accurate results.
- Accuracy: this will indicate if data entries are true to their real values. This is also important to get an accurate representation of the true values.

In [ ]:
# Count the amount of null values
print(df.isnull().sum())

The output above shows that there are many missing entries for the following (relevant) variables:
- GC (General Classification)
- Timelag
- UCI Ranking
- Points

These missing entries can, for example, be dropped, substituted, imputated, etc.

In [ ]:
inaccurate = df[df['id'] == 13323]
accurate = df[df['id'] == 216]

print("Here's an example of an inaccurate entry for 'length':\n")
print(inaccurate[['Rnk', 'Rider', 'Length']], "\n\n")
print("Here's an example of an accurate entry for 'length':\n")
print(accurate[['Rnk', 'Rider', 'Length']])

The output cell above shows an example of inaccurate/arbitrary entry that is present in the database.

### 5. Preparing the Data

#### 5.1 Actions to take
The following assessments can be made regarding the actions that need to be taken to clean the data:
- Some entries need to be uniformly formatted so they can be used for further analysis.
    - Time needs to be formatted to be correctly identified as an integer value
    - Dates need to be standardized to a certain format to be more usable with other data. It could even be categorized to determine the performance of riders per season.
    - Numerical values that are entered as strings, need to be converted from a string value to a float. For example, 'length' is formatted as a string like '150 km'. By stripping 'km' from this value will prepared it to be turned into a numercal value.
- Missing data needs to be handled. Columns with missing numerical values can be dealt with using mean or median imputation. Columns that are missing significant chunks of data need to be removed, or have rows with missing data removed.
- Extreme values should be removed by identifying outliers. 
- Data shoud be normalized.

#### 5.2 Formatting values
Dates and times can be converted to numerical values. Years and months can be useful for, for example, determining which season a rider has peak performance. Data like individual days and time of which an event took place are not as important, so they will be dropped. For example "12 Oktober 2016, 12:30" will be split into a column "Month: March" and "Year: 2016", the rest of the entry will be dropped.

The 'Time' entry, which represents the amount of time it took to complete a race, will also be converted into a single numerical value for simplicity. This will make it easier for analysis purposes, because it provides a more granular measure of time.

Additionally, some columns hold multiple values inside of a single variable in a text format. Specifically the 'pps' and 'rdr' columns in the 'riders' table hold important information on the total amount of points per type of a race an individual rider accumalated, and it also contains individual all-time rankings. These values will need to be split and put into separate columns for individual racers.

*Note that the 'riders' table was already combined with the 'race_results' table in our EDA.*

#### 5.3 Handling missing data
Missing data will be handled by mean imputation. For numeric variables, this is achieved by replacing missing values with the mean of the non-missing values withing that variable. This is done so that variables with large amounts of missing variables are still usable for analysis.
If it turns out that mean imputation will not result in accurate results, other options, like regression imputation, will also be explored. Regression imputation is performed for variables of which the outcome relies on other variables. For example, if weight is correlated to height, age and country, missing values for weight could be predicted by using the correlated values.

#### 5.4 Normalizing data
Normalizing data is a way of scaling different values so that they are on a similar scale or range. A standardized version of a variable can easily be achieved by dividing each variable by its standard deviation. The purpose of normalization is to give all variables equal importance in terms of variability.

#### 5.5 Identifying outliers
In the cells above, it was shown how using scatterplots was used to identify if certain values contained outliers. Boxplots can also be used to detect outliers in a visual manner.
To exclude extreme outliers from our analysis, computing Z-scores will be used. By calculating the distance between a data point and the mean, it can be determined if a data point is an outlier or not. 
the Interquartile Range method (IQR) will be used. This involves determining the first and third quartile of data, and then using these values to determine the boundaries for what will be considered an outlier. By employing outlier detection, it is ensured that the outcome of the data analysis is not skewed by extreme values.

### 6. Preparing Data for Machine Learning

The next step is preparing the data, so that machine learning can be performed over the dataset. This includes reducing the dimensions of the data, making text usable for analysis and applying reduction techniques. With a large dataset, the number of variables must be reduced for data mining algorithms to operate efficiently. Dimension reduction must be done before the data mining model is deployed.

The next five approaches can be taken to apply dimension reduction on a dataset:
1. Removing or combining categories based on the knowledge of the dataset.
2. Summarizing data to detect information overlap between variables. By doing this, redundant categories and or variables can be removed or combined.
3. Converting categorical variables into numerical variables, by using data conversion techniques.
4. Employing automated reduction techniques, such as Principal Component Analysis (PCA). By performing PCA, a new set of variables is created, which are weighted averages of the original variables. These new variables are uncorrelated and a small subset of these variables contains most of their combined information. By doing this, the dimensions of the dataset are reduced by using only a sybset of the new variables.
5. Data mining methods such as regression models, classification and regression trees can be used to determine redundant variables for removal and for combining similar categories into categorical variables.

**Why perform dimension reduction?**


In [ ]:
print(df['Time'])

### 7. Performing Data Cleaning

The following code cells will show how the following steps in data cleaning are performed:
1. Converting month and year into separate numerical values.
2. Converting 'Time' into single numerical values. Do the same for time lag.
3. Splitting values from 'pps' and 'rdr' and putting those values into separate columns.
4. Dropping columns that are not needed for analysis
5. Applying mean imputation for missing values.
6. Identifying outliers
7. Normalizing data

##### 7.1 Converting month and year into separate numerical values

In [ ]:
# Convert the 'Date' column to DateTime format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format='%d %B %Y')

# Remove rows where 'Date' is NaT
df = df.dropna(subset=['Date'])

# Normalize 'Date' to strip out time if it's present (this keeps just the date part)
df['Date'] = df['Date'].dt.normalize()

# Extract the month and year from the column and put them in their own columns
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

print(df.info(), "\n")

print("Unique values for 'months': ", df['Month'].unique(), "\n")
print("Unique values for 'years': ",df['Year'].unique())



##### 7.2.1 Converting 'Time' into single numerical values.

In the code cell below, we will convert the 'Time' column to time in seconds. If the time in seconds is equal to 0, it will use numpy to return NaN as a value.

In [ ]:
def time_to_seconds(time_str):
    # Remove commas and any spaces
    time_str = time_str.replace(',', '').strip()

    # Check the consistency of the time format using regular expressions
    match = re.match(r'(\d{1,2}):(\d{2}):(\d{2})', time_str)
    if match:
        hours, minutes, seconds = map(int, match.groups())
        total_seconds = hours * 3600 + minutes * 60 + seconds  # Convert to seconds
        if total_seconds == 0:
            return np.nan  # Return NaN for 0.0 seconds
        return total_seconds

    # Do the same as the loop above, but now for MM:SS format.
    match = re.match(r'(\d{1,2}):(\d{2})', time_str)
    if match:
        minutes, seconds = map(int, match.groups())
        total_seconds = minutes * 60 + seconds
        if total_seconds == 0:
            return np.nan  
        return total_seconds

    # Do the same but for 0:00, 0:01, etc.
    match = re.match(r'(\d{1,2}):(\d{1,2})', time_str)  # for formats like 0:01
    if match:
        minutes, seconds = map(int, match.groups())
        total_seconds = minutes * 60 + seconds
        if total_seconds == 0:
            return np.nan 
        return total_seconds
    
    # If format doesn't match, return NaN
    return np.nan

# Apply the conversion function to the 'Time' column
df['Time_seconds'] = df['Time'].apply(time_to_seconds)

print(df[['Time', 'Time_seconds']].head(10))
df.info()


##### 7.2.2 Converting 'Timelag' into single numerical values.

Now let's do the same for timelag, where we convert the timelag values to seconds

In [ ]:
def timelag_to_seconds(timelag_str):
    # Check if the timelag_str contains missing values
    if pd.isna(timelag_str):
        return np.nan

    # Remove the '+' sign
    timelag_str = timelag_str.lstrip('+').strip()

    match = re.match(r'(\d{1,2}):(\d{2}):(\d{2})', timelag_str)
    if match:
        hours, minutes, seconds = map(int, match.groups())
        return hours * 3600 + minutes * 60 + seconds

    # Check for MM:SS format (without hours)
    match = re.match(r'(\d{1,2}):(\d{2})', timelag_str)
    if match:
        minutes, seconds = map(int, match.groups())
        return minutes * 60 + seconds
    
    match = re.match(r'(\d+):(\d+)', timelag_str)
    if match:
        minutes, seconds = map(int, match.groups())
        return minutes * 60 + seconds

    return np.nan

df['Timelag_in_seconds'] = df['Timelag'].apply(timelag_to_seconds)

df['Timelag_in_seconds'] = df['Timelag_in_seconds'].replace(0.0, np.nan)

print(df[['Timelag', 'Timelag_in_seconds']].head(10))
df.info()


##### 7.3.1 Splitting values from 'pps' and putting those values into separate columns.


##### 7.3.2 Splitting values from 'rdr' and putting those values into separate columns.
